In [30]:
import numpy as np
import pandas as pd
import MySQLdb
import json
from time import time

In [11]:
connection = MySQLdb.connect(host='localhost', port=3306, user='root', passwd='admin', db='edxapp', charset='utf8')
data = pd.read_sql("select state  from courseware_studentmodule where course_id = 'KNU/101/2014_T2'" + \
                   " and module_id like '%e39129c796754099b0974db74c5b5b01' and grade is not null ",con=connection)

In [12]:
values = data.values.tolist()

In [68]:
result = {}
start = time()
for i in values:
    answer_info = json.loads(i[0])
    for answer, options in answer_info['student_answers'].iteritems():
        for option in options:
            name = u'{0}_{1}'.format(answer,option)
            if name in result:
                result[name] += 1
            else:
                result[name] = 1

def GetNumber(key):
    i = key.index(u'_')+1
    newkey = key[i:]
    i = newkey.index(u'_')
    return int(newkey[:i])

question_keys = {}
ids = list(set(map(GetNumber,result)))
for i, j in enumerate(ids):
    question_keys[j] = i

result2 = {}
for i,j in result.iteritems():
    testNumber = GetNumber(i)
    back_index =  i[::-1].index('_')
    choise_number = int(i[len(i)-back_index:])
    testName = question_keys[testNumber]
    if testName not in result2:
        result2[testName] = {choise_number:j}
    else:
        result2[testName][choise_number] = j

print time()-start

0.414000034332


In [69]:
result2

{0: {0: 146, 1: 3233, 2: 190, 3: 247, 4: 100, 5: 81},
 1: {0: 114, 1: 93, 2: 212, 3: 3396, 4: 57, 5: 72},
 2: {0: 188, 1: 213, 2: 2658, 3: 268, 4: 451, 5: 229},
 3: {0: 140, 1: 113, 2: 95, 3: 3482, 4: 37, 5: 66},
 4: {0: 360, 1: 268, 2: 287, 3: 2788, 4: 237, 5: 107},
 5: {0: 133, 1: 184, 2: 3313, 3: 107, 4: 106, 5: 75},
 6: {0: 246, 1: 337, 2: 188, 3: 2719, 4: 353, 5: 245},
 7: {0: 2919, 1: 587, 2: 3089, 3: 3064, 4: 476, 5: 626, 6: 361, 7: 466},
 8: {0: 317, 1: 432, 2: 172, 3: 487, 4: 2522, 5: 76},
 9: {0: 325, 1: 255, 2: 201, 3: 2814, 4: 273, 5: 129},
 10: {0: 4, 1: 30, 2: 6, 3: 26, 4: 9, 5: 68}}

In [15]:
values2 = []
for j in values[:1]:
    answer_info = json.loads(j[1])
    #print answer_info#['student_answers']
    answers = {}
    """
    for i in answer_info[u'correct_map'].keys():
        if i not in answer_info[u'student_answers'].keys():
            continue
        isCorrect = answer_info[u'correct_map'][i][u'correctness'] == u'correct'
        choise = map(lambda x: int(x[7:]),answer_info[u'student_answers'][i])
        answers[i] = {'isCorrect' : isCorrect, 'choise' : choise}
    """
    values2.append([j[0],answers])

IndexError: list index out of range

In [62]:
from pymongo import MongoClient
from xml.dom import minidom

In [58]:
client = MongoClient('localhost', 27017)
modules = client.edxapp.modulestore

In [127]:
course_id = 'KNU/101/2014_T2'
org, course, period = course_id.split('/')
problems = []
for problem in modules.find({"_id.course":course,"_id.org":org,"_id.category":"problem"}):
    name = problem['_id']['name']
    xml = minidom.parseString(problem['definition']['data']['data'].encode('utf-8'))
    vertical = modules.find_one({"_id.course":course,"_id.org":org,"_id.category":"vertical",'definition.children': {'$elemMatch' : {'$regex' : ".*{0}.*".format(name)}}})
    display_name= vertical['metadata']['display_name']
    structure = []
    for node in xml.documentElement.getElementsByTagName('checkboxgroup'):
        task =  node.getAttribute('label')[:80] + '...'
        options = [{'correct': i.getAttribute('correct') == 'true', 'text': i.firstChild.nodeValue[:80] + ('...' if len(i.firstChild.nodeValue) > 80 else '')} for i in node.getElementsByTagName('choice')]
        structure.append({'task':task, 'options':options})
    problems.append({'org':org,'course':course,'name':name, 'display_name':display_name, 'structure':structure})

problems

[{'course': '101',
  'display_name': u'\u0422\u0435\u0441\u0442 1 \u0442\u0438\u0436\u043d\u044f',
  'name': u'e39129c796754099b0974db74c5b5b01',
  'org': 'KNU',
  'structure': [{'options': [{'correct': False,
      'text': u'\u0414\u0438\u0441\u043a\u0443\u0441\u0456\u044e \u043d\u0430\u0432\u043a\u043e\u043b\u043e \u0434\u0435\u0440\u0436\u0430\u0432\u043d\u043e\u0457 \u043f\u0440\u0438\u043d\u0430\u043b\u0435\u0436\u043d\u043e\u0441\u0442\u0456 \u0411\u0443\u043a\u043e\u0432\u0438\u043d\u0438'},
     {'correct': True,
      'text': u'\u0414\u0438\u0441\u043a\u0443\u0441\u0456\u044e \u043d\u0430\u0432\u043a\u043e\u043b\u043e \u043c\u0430\u0439\u0431\u0443\u0442\u043d\u044c\u043e\u0457 \u0434\u043e\u043b\u0456 \u041a\u0430\u0440\u043f\u0430\u0442\u0441\u044c\u043a\u043e\u0457 \u0423\u043a\u0440\u0430\u0457\u043d\u0438'},
     {'correct': False,
      'text': u'\u0414\u0438\u0441\u043a\u0443\u0441\u0456\u044e \u043d\u0430\u0432\u043a\u043e\u043b\u043e \u043f\u043e\u0447\u0430\u0442\u04

In [128]:
for i in problems:
    print i['display_name']

Тест 1 тижня
Тест 2 тижня
Тест 3 тижня
Тест 4 тижня
Тест 5 тижня
Тест 6 тижня
Іспит
